<a href="https://colab.research.google.com/github/Ahtesham519/Genrative_Deep_learning_v2_2023/blob/main/cgan_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import(
    layers,
    models,
    callbacks,
    utils,
    metrics,
    optimizers,
)


#0. Parameters


In [ ]:
IMAGE_SIZE = 64
CHANNELS = 3
CLASSES = 2
BATCH_SIZE = 128
Z_DIM = 32
LEARNING_RATE = 0.00005
ADAM_BETA_1 = 0.5
ADAM_BETA_2 = 0.999
EPOCHS = 20
CRITIC_STEPS = 3
GP_WEIGHT = 10.0
LOAD_MODEL = False
ADAM_BETA_1 = 0.5
ADAM_BETA_2 = 0.9
LABEL = "Blond_Hair"

#1. Prepare the data

In [ ]:
#Load the label dataset
attributes = pd.read_csv("/app/data/celeba-dataset/list_attr_celeba.csv")
print(attributes.columns)
attributes.head()

In [ ]:
#Load the data
labels = attributes[LABEL].tolist()
int_labels = [x if x ==1 else 0 for in labels]

In [ ]:
train_data = utils.image_dataset_from_directory(
    "/app/data/celeba-dataset/img_align_celeba",
    labels = int_labels,
    color_mode = "rgb",
    image_size = (IMAGE_SIZE , IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    shuffle = True ,
    seed = 42,
    interpolation = "bilinear",
)

In [ ]:
#Preprocess the data
def preprocess(img):
  """
  Normalize and reshape the images
  """
  img = (tf.cast(img, "float32") - 127.5)/ 127.5
  return img

train = train_data.map(
    lambda x, y: (preprocess(x) , tf.one_hot(y, depth = CLASSES))
)


In [ ]:
#Show some faces from the directory training set
train_sample = sample_batch(train)



#2. Build the GAN


In [ ]:
critic_input = layers.Input(shape = (IMAGE_SIZE , IMAGE_SIZE , CHANNELS))
label_input = layers.Input(shape = (IMAGE_SIZE , IMAGE_SIZE , CLASSES))
x = layers.Concatenate(axis = -1 )([critic_input, label_input])
x = layers.Conv2D(64, kernel_size = 4, strides = 2, padding = "same")(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Conv2D(128, kernel_size = 4, strides = 2, padding = "same")(x)
x = layers.LeakyReLU()(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(128, kernel_size = 4, strides = 2, padding = "same")(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(128, kernel_size = 4, strides = 2, padding = "same")(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(1, kernel_size = 4, strides = 1 , padding = "valid")(x)
critic_output = layers.Flatten()(x)

critic = models.Model([critic_input, label_input] , critic_output)
critic.summary()


In [ ]:
generator_input  = layers.Input(shape = (Z_DIM, ))
label_input = layers.Input(shape= (CLASSES, ))
x = layers.Concatenate(axis = -1)([generator_input, label_input])
x = layers.Reshape((1, 1, Z_DIM +CLASSES))(x)
x = layers.Conv2DTranspose(
    128, kernel_size = 4, strides = 1 , padding = "valid" , use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Conv2DTranspose(
    128, kernel_size = 4, strides = 2, padding = "same", use_bias = False
)(x)
x= layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)